In [49]:
import requests
import json
import os
import pandas as pd
from functools import reduce
import operator
import pyjsonviewer
import geopandas as gpd
import time
import requests
from bs4 import BeautifulSoup

from dotenv import load_dotenv
load_dotenv()

True

## Funciones útiles

In [2]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [3]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [4]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [data["latt"], data["longt"]]}
    except:
        return data

## Lista de categorías e id de Foursquare
- scrapping

In [47]:
def get_categories():
    url = "https://developer.foursquare.com/docs/build-with-foursquare/categories/"
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    venue_cat = soup.find_all("ul", {"class": "VenueCategories__Wrapper-sc-15dn453-0 dmcDKQ"})
    cat = {}
    li = venue_cat[0].find_all("li")
    for l in li:
        cat[l.find("h3").getText()] = l.find("p").getText()
        
    return pd.DataFrame.from_dict(cat,orient="Index").reset_index().rename(columns={"index": "category", 0: "categoryId"})

categories = get_categories()

In [50]:
categories.sample(3)

,category,categoryId
6,Casino,4bf58dd8d48988d17c941735
630,Library,4bf58dd8d48988d12f941735
495,Basketball Court,4bf58dd8d48988d1e1941735


In [6]:
def categories_dic(cat_list):    
    dicc = {}
    for c in cat_list:
        dicc[c] = categories.categoryId[categories["category"] == c].values[0]
    return dicc

In [7]:
venues_list = ["Design Studio","Nursery School","Preschool","Elementary School","Tech Startup","Coffee Shop",
               "Airport Terminal","Train Station",
              "Nightlife Spot","Vegetarian / Vegan Restaurant","Basketball Stadium",
               "Veterinarian","Pet Service","Pet Store"]  

In [8]:
cat_dic = categories_dic(venues_list)
cat_dic

{'Design Studio': '4bf58dd8d48988d1f4941735',
 'Nursery School': '4f4533814b9074f6e4fb0107',
 'Preschool': '52e81612bcbc57f1066b7a45',
 'Elementary School': '4f4533804b9074f6e4fb0105',
 'Tech Startup': '4bf58dd8d48988d125941735',
 'Coffee Shop': '4bf58dd8d48988d1e0931735',
 'Airport Terminal': '4bf58dd8d48988d1eb931735',
 'Train Station': '4bf58dd8d48988d129951735',
 'Nightlife Spot': '4d4b7105d754a06376d81259',
 'Vegetarian / Vegan Restaurant': '4bf58dd8d48988d1d3941735',
 'Basketball Stadium': '4bf58dd8d48988d18b941735',
 'Veterinarian': '4d954af4a243a5684765b473',
 'Pet Service': '5032897c91d4c4b30a586d69',
 'Pet Store': '4bf58dd8d48988d100951735'}

In [9]:
def localizations_coor(dicc):
    local = {}
    for city,loc in localizations_dicc.items():
        local[city] = geocode(loc)["coordinates"]
        time.sleep(15)
    return local
    

In [10]:
localizations_dicc={"Austin":"Texas Capitol, Austin, Texas","Hamburg":"Hamburg city center,Hamburg,Germany","Newcastle":"Newcastle upon Tyne, United Kingdom"}
localizations = localizations_coor(localizations_dicc)
localizations

{'Austin': ['30.35471', '-97.69344'],
 'Hamburg': ['53.56250', '9.95730'],
 'Newcastle': ['54.96890', '-1.61764']}

In [11]:
#  localizations = {'Austin': ['30.35471', '-97.69344'],
#  'Hamburg': ['53.56250', '9.95730'],
#  'Newcastle': ['54.96890', '-1.61764']}

In [16]:
def place_store_dicc(localizations,cat_dic):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    client_id = os.getenv("four")
    client_secret = os.getenv("foursec")
    
    venues_dic = {}
    aus_dic ={}
    ham_dic = {}
    ncas_dic ={}
    for loc,coor in localizations.items():
        for cat,cat_id in cat_dic.items():
            time.sleep(1)
            parametros={
                "client_id": client_id,
                "client_secret": client_secret,
                "v": "20180323",
                "ll": f"{coor[0]}, {coor[1]}",
                "categoryId":cat_id,
                "radius":2000
            }
            if loc == "Austin":
                aus_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"] 
                if cat == "Pet Store":
                    print(f"{loc} done")
            elif loc == "Hamburg":                
                ham_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"]
                if cat == "Pet Store":
                    print(f"{loc} done")
            elif loc == "Newcastle":                
                ncas_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"] 
                if cat == "Pet Store":
                    print(f"{loc} done")
            
    venues_dic = {"Austin":aus_dic,
                 "Hamburg":ham_dic,
                  "Newcastle": ncas_dic
                 }

    return venues_dic

    
    


In [17]:
total_venues = place_store_dicc(localizations,cat_dic)

Austin done
Hamburg done
Newcastle done


In [34]:
def extraetodo(json):
    total = []
    for city,categories in json.items():
        for cat,value in categories.items():
            try:
                todo = {"nombre": ["name"],"sub_category":["categories",0,"name"],
                        "latitud": ["location", "lat"], "longitud": ["location", "lng"],
                        "distance":["location","distance"]}

                for elemento in value:
                    store = {key: getFromDict(elemento,value) for key,value in todo.items()}
                    store["location"] = type_point([store["latitud"], store["longitud"]])
                    store["city"] = city
                    store["category"] = cat
                    total.append(store)

                #print(f"{elemento} done")
            except:

                print(f"{value.index(elemento)} wrong")

    return total

In [35]:
venues = extraetodo(total_venues)

In [36]:
venues_df = pd.DataFrame(venues)

In [44]:
venues_df.sample(3)

,nombre,sub_category,latitud,longitud,distance,location,city,category
314,Zerolight,Tech Startup,54.969619,-1.604692,831,"{'type': 'Point', 'coordinates': [54.969619, -...",Newcastle,Tech Startup
187,Unikat,Hookah Bar,53.566865,9.981231,1655,"{'type': 'Point', 'coordinates': [53.566864793...",Hamburg,Nightlife Spot
380,Head of Steam,Bar,54.969520,-1.615526,151,"{'type': 'Point', 'coordinates': [54.969520290...",Newcastle,Nightlife Spot


In [38]:
#borrar todos los cafés que no sean starbucks
no_starbucks = list(venues_df.index[(venues_df["category"] == "Coffee Shop") & (venues_df["nombre"] != "Starbucks")])
venues_df.drop(no_starbucks,axis=0, inplace = True)

In [39]:
venues_df.sample(5)

,nombre,sub_category,latitud,longitud,distance,location,city,category
294,Westgate Hill Primary School,Elementary School,54.974127,-1.636699,1349,"{'type': 'Point', 'coordinates': [54.974127111...",Newcastle,Elementary School
396,Ernest,Bar,54.976620,-1.596348,1609,"{'type': 'Point', 'coordinates': [54.976620429...",Newcastle,Nightlife Spot
144,Digital Pioneers,Tech Startup,53.543908,9.947812,2162,"{'type': 'Point', 'coordinates': [53.543907503...",Hamburg,Tech Startup
279,Printing.com,Design Studio,54.967967,-1.621319,257,"{'type': 'Point', 'coordinates': [54.967967368...",Newcastle,Design Studio
80,Kita Bucerius Law School,Daycare,53.559997,9.984201,1800,"{'type': 'Point', 'coordinates': [53.559997308...",Hamburg,Nursery School


In [40]:
venues_df.to_csv("../data/venues_df.csv",index = True)

In [41]:
Austin_df = venues_df[venues_df["city"] == "Austin"].sort_values(["category","distance"])
Austin_df.head(3)

,nombre,sub_category,latitud,longitud,distance,location,city,category
0,Viatran,Design Studio,30.348400,-97.706521,1439,"{'type': 'Point', 'coordinates': [30.3484, -97...",Austin,Design Studio
1,Solid Prototype,Design Studio,30.351604,-97.710106,1637,"{'type': 'Point', 'coordinates': [30.351604461...",Austin,Design Studio
4,Mollie Barrington Elementary School,Elementary School,30.361054,-97.694413,712,"{'type': 'Point', 'coordinates': [30.361053936...",Austin,Elementary School


In [42]:
Hamburg_df = venues_df[venues_df["city"] == "Hamburg"].sort_values(["category","distance"])
Hamburg_df.head(3)

,nombre,sub_category,latitud,longitud,distance,location,city,category
54,ps145 | Büro für schöne Bekanntmachung,Design Studio,53.559864,9.956979,294,"{'type': 'Point', 'coordinates': [53.559864, 9...",Hamburg,Design Studio
47,LIGALUX,Design Studio,53.567143,9.953387,578,"{'type': 'Point', 'coordinates': [53.567143471...",Hamburg,Design Studio
59,SØNXEN Kommunikationsdesign,Design Studio,53.557445,9.961580,629,"{'type': 'Point', 'coordinates': [53.557445, 9...",Hamburg,Design Studio


In [43]:
Newcastle_df = venues_df[venues_df["city"] == "Newcastle"].sort_values(["category","distance"])
Newcastle_df.head(3)

,nombre,sub_category,latitud,longitud,distance,location,city,category
416,Northumbria University Sport Central,Athletics & Sports,54.978318,-1.606642,1262,"{'type': 'Point', 'coordinates': [54.978317593...",Newcastle,Basketball Stadium
330,Starbucks,Coffee Shop,54.969435,-1.616766,81,"{'type': 'Point', 'coordinates': [54.969435, -...",Newcastle,Coffee Shop
331,Starbucks,Coffee Shop,54.972522,-1.617541,403,"{'type': 'Point', 'coordinates': [54.972522, -...",Newcastle,Coffee Shop
